# **[Deploy a Machine Learning Model to a Real-Time Inference Endpoint](https://aws.amazon.com/tutorials/machine-learning-tutorial-deploy-model-to-real-time-inference-endpoint/)** 

Amazon SageMaker를 사용하면 콘솔을 사용하여 시각적으로 모델을 배포하거나 SageMaker Studio 또는 SageMaker 노트북을 사용하여 프로그래밍 방식으로 모델을 배포할 수 있습니다. 이 튜토리얼에서는 SageMaker Studio 도메인이 필요한 SageMaker Studio 노트북을 사용하여 프로그래밍 방식으로 모델을 배포합니다. 

AWS 계정에는 AWS 리전당 여러 개의 Studio 도메인을 가질 수 있습니다. 미국 동부(버지니아 북부, us-east-1) 지역에 이미 SageMaker Studio 도메인이 있는 경우 SageMaker Studio 설정 가이드에 따라 필요한 AWS IAM 정책을 SageMaker Studio 계정에 첨부한 다음 바로 진행합니다. 

기존 SageMaker Studio 도메인이 없는 경우 SageMaker Studio 도메인을 생성하고 이 자습서의 나머지 부분에 필요한 권한을 추가하는 [AWS CloudFormation](https://us-east-1.console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/quickcreate?templateURL=https://sagemaker-sample-files.s3.amazonaws.com/libraries/sagemaker-user-journey-tutorials/CFN-SM-IM-Lambda-catalog.yaml&stackName=CFN-SM-IM-Lambda-Catalog) 템플릿을 실행합니다.

CloudFromation 스택이름을 **CFN-SM-IM-Lambda-Catalog** 로 변경하지 마시고 10분정도 기다리시면 **SageMaker > studio > StudioDomain 선택후 studio-user** 로 시작합니다. 자세한 안내는 [Deploy a Machine Learning Model to a Real-Time Inference Endpoint](https://aws.amazon.com/tutorials/machine-learning-tutorial-deploy-model-to-real-time-inference-endpoint/) 참고하세요.  

In [30]:
%pip install --upgrade -q aiobotocore

Note: you may need to restart the kernel to use updated packages.


In [31]:
import pandas as pd
import numpy as np
import boto3
import sagemaker
import time
import json
import io
from io import StringIO
import base64
import pprint
import re

from sagemaker.image_uris import retrieve

sess = sagemaker.Session()
write_bucket = sess.default_bucket()
write_prefix = "fraud-detect-demo"

region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtime_client = boto3.client("sagemaker-runtime")
sm_autoscaling_client = boto3.client("application-autoscaling")

sagemaker_role = sagemaker.get_execution_role()


# S3 locations used for parameterizing the notebook run
read_bucket = "sagemaker-sample-files"
read_prefix = "datasets/tabular/synthetic_automobile_claims" 
model_prefix = "models/xgb-fraud"

data_capture_key = f"{write_prefix}/data-capture"

# S3 location of trained model artifact
model_uri = f"s3://{read_bucket}/{model_prefix}/fraud-det-xgb-model.tar.gz"

# S3 path where data captured at endpoint will be stored
data_capture_uri = f"s3://{write_bucket}/{data_capture_key}"

# S3 location of test data
test_data_uri = f"s3://{read_bucket}/{read_prefix}/test.csv" 

In [32]:
print(f' sagemaker_role : {sagemaker_role}')

 sagemaker_role : arn:aws:iam::720025708473:role/CFN-SM-IM-Lambda-Catalog-SageMakerExecutionRole-XRg8SB5ffteM


SageMaker에서는 학습된 모델을 실시간 추론 엔드포인트에 배포하는 여러 가지 방법이 있습니다: [SageMaker SDK](https://sagemaker.readthedocs.io/en/stable/v2.html), [AWS SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html) - Boto3 및 SageMaker 콘솔. 자세한 내용은 Amazon SageMaker 개발자 가이드에서 [추론을 위한 모델 배포하기](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html)를 참조하세요. SageMaker SDK는 AWS SDK - Boto3에 비해 추상화가 더 많으며, 후자는 모델 배포를 더 잘 제어할 수 있도록 하위 수준 API를 노출합니다. 이 튜토리얼에서는 AWS SDK -Boto3를 사용하여 모델을 배포합니다. 모델을 배포하려면 다음 세 단계를 순서대로 따라야 합니다:

1. 모델 아티팩트에서 SageMaker 모델 만들기
2. 인스턴스 유형 및 개수를 포함한 속성을 지정하는 엔드포인트 구성(endpoint configuration)을 생성합니다.
3. 엔드포인트 구성을 사용하여 엔드포인트(endpoint) 생성

In [33]:
# Retrieve the SageMaker managed XGBoost image
training_image = retrieve(framework="xgboost", region=region, version="1.3-1")

# Specify a unique model name that does not exist
model_name = "fraud-detect-xgb"
primary_container = {
                     "Image": training_image,
                     "ModelDataUrl": model_uri
                    }

model_matches = sm_client.list_models(NameContains=model_name)["Models"]
if not model_matches:
    model = sm_client.create_model(ModelName=model_name,
                                   PrimaryContainer=primary_container,
                                   ExecutionRoleArn=sagemaker_role)
else:
    print(f"Model with name {model_name} already exists! Change model name to create new")

SageMaker 모델이 생성된 후, 다음 코드를 복사하여 붙여넣기하여 **Boto3 create_endpoint_config** 메서드를 사용하여 엔드포인트를 구성합니다. **create_endpoint_config** 메서드의 주요 입력은 엔드포인트 구성 이름과 추론 인스턴스 유형 및 수와 같은 변형 정보, 배포할 모델의 이름, 엔드포인트가 처리해야 하는 트래픽 공유입니다. 이러한 설정과 함께 **DataCaptureConfig**를 지정하여 데이터 캡처를 설정할 수도 있습니다. 이 기능을 사용하면 실시간 엔드포인트가 요청 및/또는 응답을 캡처하여 Amazon S3에 저장하도록 구성할 수 있습니다. 데이터 캡처는 모델 모니터링을 설정하는 단계 중 하나이며, 기준 메트릭 및 모니터링 작업과 결합하면 테스트 데이터 메트릭과 기준선을 비교하여 [모델 성능을 모니터링](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html)하는 데 도움이 됩니다. 이러한 모니터링은 모델 또는 데이터 드리프트 및 감사 목적에 따라 모델 재교육을 예약하는 데 유용합니다. 현재 설정에서는 입력(수신 테스트 데이터)과 출력(모델 예측)이 모두 캡처되어 기본 S3 버킷에 저장됩니다.

In [34]:
# Endpoint Config name
endpoint_config_name = f"{model_name}-endpoint-config"

# Endpoint config parameters
production_variant_dict = {
                           "VariantName": "Alltraffic",
                           "ModelName": model_name,
                           "InitialInstanceCount": 1,
                           "InstanceType": "ml.m5.xlarge",
                           "InitialVariantWeight": 1
                          }

# Data capture config parameters
data_capture_config_dict = {
                            "EnableCapture": True,
                            "InitialSamplingPercentage": 100,
                            "DestinationS3Uri": data_capture_uri,
                            "CaptureOptions": [{"CaptureMode" : "Input"}, {"CaptureMode" : "Output"}]
                           }


# Create endpoint config if one with the same name does not exist
endpoint_config_matches = sm_client.list_endpoint_configs(NameContains=endpoint_config_name)["EndpointConfigs"]
if not endpoint_config_matches:
    endpoint_config_response = sm_client.create_endpoint_config(
                                                                EndpointConfigName=endpoint_config_name,
                                                                ProductionVariants=[production_variant_dict],
                                                                DataCaptureConfig=data_capture_config_dict
                                                               )
else:
		print(f"Endpoint config with name {endpoint_config_name} already exists! Change endpoint config name to create new")

Endpoint config with name fraud-detect-xgb-endpoint-config already exists! Change endpoint config name to create new


다음 코드를 복사하여 붙여넣어 엔드포인트를 만듭니다. create_endpoint 메서드는 엔드포인트 구성을 매개변수로 받아 엔드포인트 구성에 지정된 모델을 컴퓨트 인스턴스에 배포합니다. 모델을 배포하는 데 약 6분이 소요됩니다.


In [35]:
endpoint_name = f"{model_name}-endpoint"

endpoint_matches = sm_client.list_endpoints(NameContains=endpoint_name)["Endpoints"]
if not endpoint_matches:
    endpoint_response = sm_client.create_endpoint(
                                                  EndpointName=endpoint_name,
                                                  EndpointConfigName=endpoint_config_name
                                                 )
else:
    print(f"Endpoint with name {endpoint_name} already exists! Change endpoint name to create new")

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
while status == "Creating":
    print(f"Endpoint Status: {status}...")
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
print(f"Endpoint Status: {status}")

Endpoint with name fraud-detect-xgb-endpoint already exists! Change endpoint name to create new
Endpoint Status: InService


엔드포인트 상태가 서비스 중으로 변경된 후 REST API, AWS SDK - Boto3, SageMaker Studio, AWS CLI 또는 SageMaker Python SDK를 사용하여 엔드포인트를 호출할 수 있습니다. 이 튜토리얼에서는 AWS SDK - Boto3를 사용합니다. 엔드포인트를 호출하기 전에 직렬화 및 역직렬화를 사용하여 테스트 데이터를 엔드포인트에 적합하게 포맷하는 것이 중요합니다. 직렬화는 .csv와 같은 형식의 원시 데이터를 엔드포인트에서 사용할 수 있는 바이트 스트림으로 변환하는 프로세스입니다. 역직렬화는 바이트 스트림을 사람이 읽을 수 있는 형식으로 변환하는 역방향 프로세스입니다. 이 튜토리얼에서는 테스트 데이터 세트에서 처음 5개의 샘플을 전송하여 엔드포인트를 호출합니다. 엔드포인트를 호출하고 예측 결과를 얻으려면 다음 코드를 복사하여 붙여넣으세요. 엔드포인트(테스트 데이터 세트)에 대한 요청은 .csv 형식이므로, 페이로드를 생성하기 위해 csv 직렬화 프로세스가 사용됩니다. 그런 다음 응답을 예측 배열로 역직렬화합니다. 실행이 완료되면 셀은 모델 예측과 테스트 샘플에 대한 실제 레이블을 반환합니다. XGBoost 모델은 실제 클래스 레이블 대신 확률을 반환한다는 점에 주목하세요. 이 모델은 테스트 샘플이 허위 청구일 가능성이 매우 낮다고 예측했으며, 예측은 실제 레이블과 일치합니다.


In [36]:
# Fetch test data to run predictions with the endpoint
test_df = pd.read_csv(test_data_uri)

# For content type text/csv, payload should be a string with commas separating the values for each feature
# This is the inference request serialization step
# CSV serialization
csv_file = io.StringIO()
test_sample = test_df.drop(["fraud"], axis=1).iloc[:5]
test_sample.to_csv(csv_file, sep=",", header=False, index=False)
payload = csv_file.getvalue()
response = sm_runtime_client.invoke_endpoint(
                                             EndpointName=endpoint_name,
                                             Body=payload,
                                             ContentType="text/csv",
                                             Accept="text/csv"
                                            )

# This is the inference response deserialization step
# This is a bytes object
result = response["Body"].read()
# Decoding bytes to a string
result = result.decode("utf-8")
# Converting to list of predictions
result = re.split(",|\n",result)

prediction_df = pd.DataFrame()
prediction_df["Prediction"] = result[:5]
prediction_df["Label"] = test_df["fraud"].iloc[:5].values
prediction_df 


,Prediction,Label
0,0.02243666537106037,0
1,0.02243666537106037,0
2,0.0799826979637146,0
3,0.13931523263454437,0
4,0.03112351894378662,0


In [37]:
test_df.head()

,fraud,num_vehicles_involved,num_injuries,num_witnesses,police_report_available,injury_claim,vehicle_claim,total_claim_amount,incident_month,incident_day,...,authorities_contacted_ambulance,policy_state_ca,policy_state_az,policy_state_nv,policy_state_id,policy_state_wa,policy_state_or,customer_gender_other,customer_gender_male,customer_gender_female
0,0,2,0,0,1,58400,14247.766867,72647.766867,7,29,...,0,1,0,0,0,0,0,0,0,1
1,0,2,0,2,0,11500,10675.671347,22175.671347,11,28,...,0,1,0,0,0,0,0,0,0,1
2,0,2,0,0,1,18500,10202.266354,28702.266354,1,6,...,0,1,0,0,0,0,0,0,1,0
3,0,1,0,0,0,16300,9338.348066,25638.348066,9,1,...,0,1,0,0,0,0,0,0,1,0
4,0,2,1,0,0,14700,28145.924994,42845.924994,10,23,...,1,0,0,0,0,0,1,0,1,0


데이터 캡처는 엔드포인트 구성에서 설정되었으므로 엔드포인트의 응답과 함께 어떤 페이로드가 엔드포인트로 전송되었는지 검사할 수 있는 방법이 있습니다. 캡처한 데이터가 S3에 완전히 업로드되는 데는 다소 시간이 걸립니다. 다음 코드를 복사하여 붙여넣어 데이터 캡처가 완료되었는지 확인하세요.


In [38]:
from sagemaker.s3 import S3Downloader
print("Waiting for captures to show up", end="")
for _ in range(90):
    capture_files = sorted(S3Downloader.list(f"{data_capture_uri}/{endpoint_name}"))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
        capture_record = json.loads(capture_file[0])
        if "inferenceId" in capture_record["eventMetadata"]:
            break
    print(".", end="", flush=True)
    time.sleep(1)
print()
print(f"Found {len(capture_files)} Data Capture Files:")

Waiting for captures to show up..........................................................................................
Found 26 Data Capture Files:


캡처된 데이터는 각 엔드포인트 호출에 대해 각 줄이 JSON 값인 구조화된 데이터를 저장하는 새줄로 구분된 형식인 JSON 줄로 S3에 별도의 파일로 저장됩니다. 다음 코드를 복사하여 붙여넣어 데이터 캡처 파일을 검색하세요. 


In [40]:
capture_files = sorted(S3Downloader.list(f"{data_capture_uri}/{endpoint_name}"))
capture_file = S3Downloader.read_file(capture_files[0]).split("\n")
capture_record = json.loads(capture_file[0])
capture_record

{'captureData': {'endpointInput': {'observedContentType': 'text/csv',
   'mode': 'INPUT',
   'data': 'MiwwLDAsMSw1ODQwMCwxNDI0Ny43NjY4NjY2NzA4NSw3MjY0Ny43NjY4NjY2NzA4NSw3LDI5LDAsMTEsNDEsMTA4LDAsMSw3NTAsMzAwMCwwLDIsMjAxNCwwLDAsMSwwLDAsMSwwLDAsMCwxLDAsMCwwLDAsMSwwLDAsMSwwLDEsMCwwLDAsMCwwLDAsMCwxCjIsMCwyLDAsMTE1MDAsMTA2NzUuNjcxMzQ2NzM4MTU4LDIyMTc1LjY3MTM0NjczODE2LDExLDI4LDMsMTYsNTQsMTcxLDAsMSw3NTAsMjc1MCwyLDMsMjAxNSwwLDAsMCwwLDEsMSwwLDAsMCwxLDAsMCwwLDAsMSwwLDAsMSwwLDEsMCwwLDAsMCwwLDAsMCwxCjIsMCwwLDEsMTg1MDAsMTAyMDIuMjY2MzUzOTk5Nzc2LDI4NzAyLjI2NjM1Mzk5OTc3LDEsNiw2LDgsNTEsMTk2LDAsMSw3NTAsMzAwMCwwLDEsMjAxMSwwLDAsMCwwLDEsMSwwLDAsMSwwLDAsMCwwLDEsMCwwLDAsMSwwLDEsMCwwLDAsMCwwLDAsMSwwCjEsMCwwLDAsMTYzMDAsOTMzOC4zNDgwNjY0MzU3MTgsMjU2MzguMzQ4MDY2NDM1NzIsOSwxLDYsMyw1NywxMjIsMCwxLDc1MCwzMDAwLDEsMSwyMDE1LDAsMCwwLDEsMCwwLDEsMCwwLDAsMSwwLDAsMCwxLDAsMCwxLDAsMSwwLDAsMCwwLDAsMCwxLDAKMiwxLDAsMCwxNDcwMCwyODE0NS45MjQ5OTQxODIyNSw0Mjg0NS45MjQ5OTQxODIyNSwxMCwyMywyLDE1LDU5LDE0MiwwLDEsNzUwLDMwMDAsMiwzLDIwMTksMCwwLD

다음 코드를 복사하여 붙여넣어 base64를 사용하여 캡처된 파일의 데이터를 디코딩합니다. 이 코드는 페이로드로 전송된 5개의 테스트 샘플과 그 예측값을 검색합니다. 이 기능은 모델 응답으로 엔드포인트 부하를 검사하고 모델 성능을 모니터링하는 데 유용합니다.


In [41]:
input_data = capture_record["captureData"]["endpointInput"]["data"]
output_data = capture_record["captureData"]["endpointOutput"]["data"]
input_data_list = base64.b64decode(input_data).decode("utf-8").split("\n")
print(input_data_list)
output_data_list = base64.b64decode(output_data).decode("utf-8").split("\n")
print(output_data_list)

['2,0,0,1,58400,14247.76686667085,72647.76686667085,7,29,0,11,41,108,0,1,750,3000,0,2,2014,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1', '2,0,2,0,11500,10675.671346738158,22175.67134673816,11,28,3,16,54,171,0,1,750,2750,2,3,2015,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1', '2,0,0,1,18500,10202.266353999776,28702.26635399977,1,6,6,8,51,196,0,1,750,3000,0,1,2011,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0', '1,0,0,0,16300,9338.348066435718,25638.34806643572,9,1,6,3,57,122,0,1,750,3000,1,1,2015,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0', '2,1,0,0,14700,28145.92499418225,42845.92499418225,10,23,2,15,59,142,0,1,750,3000,2,3,2019,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0', '']
['0.02243666537106037', '0.02243666537106037', '0.0799826979637146', '0.13931523263454437', '0.03112351894378662', '']


실시간 추론 엔드포인트를 사용하는 워크로드는 일반적으로 지연 시간이 짧아야 합니다. 또한 트래픽이 급증하면 실시간 추론 엔드포인트에서 CPU 과부하, 높은 지연 시간 또는 시간 초과가 발생할 수 있습니다. 따라서 짧은 지연 시간으로 트래픽 변화를 효율적으로 처리할 수 있도록 용량을 확장하는 것이 중요합니다. SageMaker 추론 자동 확장은 워크로드를 모니터링하고 인스턴스 수를 동적으로 조정하여 저렴한 비용으로 안정적이고 예측 가능한 엔드포인트 성능을 유지합니다. 워크로드가 증가하면 자동 확장은 더 많은 인스턴스를 온라인 상태로 전환하고, 워크로드가 감소하면 불필요한 인스턴스를 제거하여 컴퓨팅 비용을 절감할 수 있습니다. 이 튜토리얼에서는 AWS SDK - Boto3를 사용하여 엔드포인트에 자동 확장을 설정합니다. SageMaker는 목표 추적 확장, 단계 확장, 주문형 확장 및 예약 확장 등 여러 유형의 자동 확장을 제공합니다. 이 자습서에서는 [선택한 확장 메트릭(target tracking scaling policy)](https://docs.aws.amazon.com/autoscaling/application/APIReference/API_TargetTrackingScalingPolicyConfiguration.html)이 선택한 목표 임계값 이상으로 증가하면 트리거되는 목표 추적 확장 정책을 사용합니다.


In [42]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)

# SageMaker expects resource id to be provided with the following structure
resource_id = f"endpoint/{endpoint_name}/variant/{resp['ProductionVariants'][0]['VariantName']}"

# Scaling configuration
scaling_config_response = sm_autoscaling_client.register_scalable_target(
                                                          ServiceNamespace="sagemaker",
                                                          ResourceId=resource_id,
                                                          ScalableDimension="sagemaker:variant:DesiredInstanceCount", 
                                                          MinCapacity=1,
                                                          MaxCapacity=2
                                                        )

다음 코드를 복사하여 붙여넣어 스케일링 정책을 만듭니다. 선택한 스케일링 메트릭은 모델 변형에 대한 각 추론 인스턴스가 호출되는 분당 평균 횟수인 **SageMakerVariantInvocationsPerInstance**입니다. 이 수치가 선택한 임계값인 5를 초과하면 자동 스케일링이 트리거됩니다.


In [43]:
# Create Scaling Policy
policy_name = f"scaling-policy-{endpoint_name}"
scaling_policy_response = sm_autoscaling_client.put_scaling_policy(
                                                PolicyName=policy_name,
                                                ServiceNamespace="sagemaker",
                                                ResourceId=resource_id,
                                                ScalableDimension="sagemaker:variant:DesiredInstanceCount",
                                                PolicyType="TargetTrackingScaling",
                                                TargetTrackingScalingPolicyConfiguration={
                                                    "TargetValue": 5.0, # Target for avg invocations per minutes
                                                    "PredefinedMetricSpecification": {
                                                        "PredefinedMetricType": "SageMakerVariantInvocationsPerInstance",
                                                    },
                                                    "ScaleInCooldown": 600, # Duration in seconds until scale in
                                                    "ScaleOutCooldown": 60 # Duration in seconds between scale out
                                                }
                                            )

다음 코드를 복사하여 붙여넣어 스케일링 정책 세부 정보를 검색합니다.


In [44]:
response = sm_autoscaling_client.describe_scaling_policies(ServiceNamespace="sagemaker")

pp = pprint.PrettyPrinter(indent=4, depth=4)
for i in response["ScalingPolicies"]:
    pp.pprint(i["PolicyName"])
    print("")
    if("TargetTrackingScalingPolicyConfiguration" in i):
        pp.pprint(i["TargetTrackingScalingPolicyConfiguration"])

'scaling-policy-DeepSeek-Coder-V2-Instruct-2024-08-26-18-13-10-225'

{   'PredefinedMetricSpecification': {   'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance'},
    'ScaleInCooldown': 600,
    'ScaleOutCooldown': 60,
    'TargetValue': 5.0}
'scaling-policy-fraud-detect-xgb-endpoint'

{   'PredefinedMetricSpecification': {   'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance'},
    'ScaleInCooldown': 600,
    'ScaleOutCooldown': 60,
    'TargetValue': 5.0}


다음 코드를 복사하여 붙여넣어 엔드포인트를 스트레스 테스트합니다. 이 코드는 250초 동안 실행되며 테스트 데이터 세트에서 무작위로 선택된 샘플을 전송하여 엔드포인트를 반복적으로 호출합니다.


In [46]:
request_duration = 250
end_time = time.time() + request_duration
print(f"Endpoint will be tested for {request_duration} seconds")
while time.time() < end_time:
    csv_file = io.StringIO()
    test_sample = test_df.drop(["fraud"], axis=1).iloc[[np.random.randint(0, test_df.shape[0])]]
    test_sample.to_csv(csv_file, sep=",", header=False, index=False)
    payload = csv_file.getvalue()
    response = sm_runtime_client.invoke_endpoint(
                                                 EndpointName=endpoint_name,
                                                 Body=payload,
                                                 ContentType="text/csv"
                                                )

Endpoint will be tested for 250 seconds


In [47]:
# print(endpoint_name)
# test_df.shape[0]
# test_sample.info()

In [48]:
test_sample.head()

,num_vehicles_involved,num_injuries,num_witnesses,police_report_available,injury_claim,vehicle_claim,total_claim_amount,incident_month,incident_day,incident_dow,...,authorities_contacted_ambulance,policy_state_ca,policy_state_az,policy_state_nv,policy_state_id,policy_state_wa,policy_state_or,customer_gender_other,customer_gender_male,customer_gender_female
171,2,2,2,0,12900,32146.55667,45046.55667,1,6,6,...,0,0,0,0,0,1,0,0,1,0


엔드포인트가 증가된 페이로드를 수신하면 아래 코드를 실행하여 엔드포인트의 상태를 확인할 수 있습니다. 이 코드는 엔드포인트의 상태가 서비스 중에서 업데이트 중으로 변경되는 시점을 확인하고 인스턴스 수를 추적합니다. 몇 분 후 상태가 서비스 중에서 업데이트 중으로, 다시 서비스 중으로 변경되었지만 인스턴스 수가 더 많아진 것을 확인할 수 있습니다.


In [49]:
# Check the instance counts after the endpoint gets more load
response = sm_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_status = response["EndpointStatus"]
request_duration = 250
end_time = time.time() + request_duration
print(f"Waiting for Instance count increase for a max of {request_duration} seconds. Please re run this cell in case the count does not change")
while time.time() < end_time:
    response = sm_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = response["EndpointStatus"]
    instance_count = response["ProductionVariants"][0]["CurrentInstanceCount"]
    print(f"Status: {endpoint_status}")
    print(f"Current Instance count: {instance_count}")
    if (endpoint_status=="InService") and (instance_count>1):
        break
    else:
        time.sleep(15)

Waiting for Instance count increase for a max of 250 seconds. Please re run this cell in case the count does not change
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 2
Status: InService
Current Instance count: 2


노트북에서 다음 코드 블록을 실행하여 이 튜토리얼에서 만든 모델, 엔드포인트 구성 및 엔드포인트를 삭제합니다. 엔드포인트를 삭제하지 않으면 엔드포인트에서 실행 중인 컴퓨팅 인스턴스에 대한 요금이 계정에 계속 누적됩니다.


In [ ]:
# Delete model
sm_client.delete_model(ModelName=model_name)

# Delete endpoint configuration
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

# Delete endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)


## **Conclusion**
축하합니다! 실시간 추론 엔드포인트에 머신 러닝 모델 배포 자습서를 마쳤습니다.
 
이 튜토리얼에서는 SageMaker 모델을 생성하고 실시간 추론 엔드포인트에 배포했습니다. AWS SDK-Boto3 API를 사용하여 엔드포인트를 호출하고 샘플 추론을 실행하여 테스트하는 동시에 데이터 캡처 기능을 활용하여 엔드포인트 페이로드와 응답을 Amazon S3에 저장했습니다. 마지막으로, 트래픽 변동을 처리하기 위해 대상 엔드포인트 호출 메트릭을 사용하여 자동 확장을 구성했습니다.
 
아래의 다음 단계 섹션에 따라 SageMaker로 머신 러닝 여정을 계속 진행할 수 있습니다.